In [1]:
!pip install --upgrade wandb

In [2]:
!pip install python-dotenv

In [3]:
from dotenv import load_dotenv
import os

# Load biến môi trường từ file .env
load_dotenv()

# Lấy key từ biến môi trường
wandb_api_key = os.getenv("WANDB_API_KEY")
print(wandb_api_key[:5])

c8767


In [4]:
import wandb
import os

# Lấy API key từ biến môi trường và đăng nhập
wandb.login(key=os.getenv("WANDB_API_KEY"))


wandb: Currently logged in as: doanngoccuong (doanngoccuong_nh). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
import os
import json
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import wandb

# 1. Tải mô hình từ artifact trên WandB
run = wandb.init(project="bert-intent-classification")  # Tên dự án trong WandB
artifact = run.use_artifact('doanngoccuong_nh/bert-intent-classification/last_model_epoch_30:v0', type='model')
artifact_dir = artifact.download()
print("Files in artifact_dir:", os.listdir(artifact_dir))

# Đường dẫn tệp cấu hình
config_path = os.path.join(artifact_dir, "config.json")

# Kiểm tra và cập nhật tệp config.json
config = {
    "model_type": "bert",
    "hidden_size": 768,
    "num_attention_heads": 12,
    "num_hidden_layers": 12,
    "vocab_size": 30522
}
with open(config_path, "w") as f:
    json.dump(config, f, indent=4)
print(f"Config.json updated at {config_path}")

# Tải tokenizer từ mô hình gốc
original_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Lưu các tệp cần thiết vào artifact_dir
original_tokenizer.save_pretrained(artifact_dir)

print(f"Tokenizer files saved to {artifact_dir}")

# 4. Tải mô hình đã lưu và tokenizer
model_path = artifact_dir  # Đường dẫn đến mô hình đã tải
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Chuyển mô hình sang chế độ đánh giá
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print(f"Model loaded and running on device: {device}")

# Bước 2: Chuẩn bị tokenizer và token hóa dữ liệu
max_seq_length = 512

def preprocess_input(question, answer, tokenizer, max_seq_length):
    """
    Tiền xử lý dữ liệu đầu vào bằng cách ghép nối câu hỏi và câu trả lời với các token đặc biệt.
    """
    input_text = f"[CLS] {question.strip()} [SEP] {answer.strip()} [SEP]"
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=max_seq_length
    )
    return inputs

# 5. Xử lý đầu vào
question = "What is the weather like today?"
answer = ""
inputs = preprocess_input(question, answer, tokenizer, max_seq_length)

# Chuyển đầu vào sang thiết bị phù hợp
inputs = {key: value.to(device) for key, value in inputs.items()}

# 6. Thực hiện dự đoán
with torch.no_grad():
    outputs = model(**inputs)  # Truyền đầu vào qua mô hình
    logits = outputs.logits  # Lấy logits từ đầu ra của mô hình
    predicted_class = torch.argmax(logits, dim=1).item()  # Lấy nhãn dự đoán

# 7. Mapping nhãn dự đoán sang tên nhãn
label_mapping = {
    0: "intent_fallback",
    1: "intent_learn_more",
    2: "intent_negative",
    3: "intent_neutral",
    4: "intent_positive",
    5: "silence"
}
predicted_label = label_mapping.get(predicted_class, f"Unknown (Class ID: {predicted_class})")

# 8. In kết quả dự đoán
print(f"Question: {question}")
print(f"Answer: {answer}")
print(f"Predicted class ID: {predicted_class}")
print(f"Predicted label: {predicted_label}")



# Kết thúc phiên WandB
wandb.finish()


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


wandb: Downloading large artifact last_model_epoch_30:v0, 419.95MB. 2 files... 
wandb:   2 of 2 files downloaded.  
Done. 0:0:1.9


Files in artifact_dir: ['tokenizer_config.json', 'special_tokens_map.json', 'model.safetensors', 'config.json', 'training_args.bin', 'vocab.txt', 'tokenizer.json']
Config.json updated at /content/artifacts/last_model_epoch_30:v0/config.json


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Tokenizer files saved to /content/artifacts/last_model_epoch_30:v0


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /content/artifacts/last_model_epoch_30:v0 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded and running on device: cpu
Question: What is the weather like today?
Answer: 
Predicted class ID: 1
Predicted label: intent_learn_more


In [7]:
# Bước 2: Chuẩn bị tokenizer và token hóa dữ liệu
max_seq_length = 512

def preprocess_input(question, answer, tokenizer, max_seq_length):
    """
    Tiền xử lý dữ liệu đầu vào bằng cách ghép nối câu hỏi và câu trả lời với các token đặc biệt.
    """
    input_text = f"[CLS] {question.strip()} [SEP] {answer.strip()} [SEP]"
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=max_seq_length
    )
    return inputs

# 5. Xử lý đầu vào
question = "Cậu có thể kể tên một hành động bắt đầu bằng từ 'play' không?"
answer = ""
inputs = preprocess_input(question, answer, tokenizer, max_seq_length)

# Chuyển đầu vào sang thiết bị phù hợp
inputs = {key: value.to(device) for key, value in inputs.items()}

# 6. Thực hiện dự đoán
with torch.no_grad():
    outputs = model(**inputs)  # Truyền đầu vào qua mô hình
    logits = outputs.logits  # Lấy logits từ đầu ra của mô hình
    predicted_class = torch.argmax(logits, dim=1).item()  # Lấy nhãn dự đoán

# 7. Mapping nhãn dự đoán sang tên nhãn
label_mapping = {
    0: "intent_fallback",
    1: "intent_learn_more",
    2: "intent_negative",
    3: "intent_neutral",
    4: "intent_positive",
    5: "silence"
}


predicted_label = label_mapping.get(predicted_class, f"Unknown (Class ID: {predicted_class})")

# 8. In kết quả dự đoán
print(f"Question: {question}")
print(f"Answer: {answer}")
print(f"Predicted class ID: {predicted_class}")
print(f"Predicted label: {predicted_label}")



Question: Cậu có thể kể tên một hành động bắt đầu bằng từ 'play' không?
Answer: 
Predicted class ID: 1
Predicted label: intent_learn_more
